In [1]:
!pip install --upgrade table
!pip install eli5

Requirement already up-to-date: table in /usr/local/lib/python3.6/dist-packages (0.0.4)


In [7]:
import pandas as pd
!pip install numpy==1.15.0
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


Wczytywanie danych

In [2]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [5]:
ls

car.h5


In [8]:
df = pd.read_hdf('car.h5')
df.shape

(106494, 155)

In [9]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

## Dummy Model

In [10]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [13]:
feats = ['car_id']
X = df[ ['car_id'] ].values
y = df['price_value'].values

model = DummyRegressor()
model.fit(X, y)
y_pred = model.predict(X)

mae(y, y_pred)

39465.934630440985

In [14]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [16]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [17]:
df['price_currency'].value_counts(normalize = True) * 100

PLN    99.80844
EUR     0.19156
Name: price_currency, dtype: float64

In [18]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [0]:
df = df[ df['price_currency'] != 'EUR' ]

In [22]:
df['price_currency'].value_counts()

PLN    106290
Name: price_currency, dtype: int64

In [23]:
df.shape

(106290, 155)

Features

In [27]:
df['param_color'].factorize()[0]

array([-1, -1, -1, ..., -1, -1, -1])

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat: #jeśli cecha zawiera końcówkę _cat, nadpisz wartość
    df[feat] = factorized_values
  else:
      df[feat + SUFFIX_CAT] = factorized_values

In [32]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]#nie chcemy tego gdy wartościa jest liczba, czyli w kolumnach z ceną
len(cat_feats)

151

In [36]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring = 'neg_mean_absolute_error')
np.mean(scores)


-19566.588937368324

In [41]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X, y)

#ktore cechy sa istotne
inp= PermutationImportance(m).fit(X, y)
eli5.show_weights(inp, feature_names = cat_feats)

Weight,Feature
0.2548 ± 0.0085,param_napęd__cat
0.2035 ± 0.0066,param_faktura-vat__cat
0.1915 ± 0.0042,param_stan__cat
0.1436 ± 0.0063,param_rok-produkcji__cat
0.0641 ± 0.0049,param_moc__cat
0.0415 ± 0.0023,param_skrzynia-biegów__cat
0.0414 ± 0.0016,feature_kamera-cofania__cat
0.0271 ± 0.0035,param_marka-pojazdu__cat
0.0194 ± 0.0022,param_pojemność-skokowa__cat
0.0163 ± 0.0004,feature_bluetooth__cat
